# Visualise results from model trained in clustering_analysis_training.ipynb

Author: Katharine Leney, April 2025

In [1]:
import joblib
import pandas as pd
import pickle
import os
import sys
import plotly.express as px  
sys.path.append("../src")
from utils.filter_dataframe import filter_dataframe_by_year, get_top_topics
from utils.parse_model_folder import parse_model_folder
from utils.generate_clean_labels import generate_clean_labels
from utils.generate_opportunity_table import generate_opportunity_table

In [2]:
# ====================================
# Load model and associated data
# ====================================

# Model name
model_name = f"bertopic_cluster_model"

# Build correct path
model_path = os.path.join("../models/", model_name, "model")

# Load the model
topic_model = joblib.load(model_path)

# Load texts and timestamps
with open(os.path.join("../models/", model_name, "texts.pkl"), "rb") as f:
    texts = pickle.load(f)

with open(os.path.join("../models/", model_name, "timestamps.pkl"), "rb") as f:
    timestamps = pickle.load(f)

model_label = parse_model_folder(model_name)
print(model_label)

# Make sure the outputs folder exists
os.makedirs("../outputs", exist_ok=True)

/Users/katharine/anaconda3/envs/iata/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Explore top topics
#topic_model.get_topic_info().head(10)
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,595,-1_fuel_emissions_passenger_global,"[fuel, emissions, passenger, global, cargo, ai...",[Region 2023 2022 5-year average (2019-2023) A...
1,0,233,0_safety_iosa_accidents_accident,"[safety, iosa, accidents, accident, audit, rat...",[19 SAFETY Safety audits Airlines on the IOSA ...
2,1,194,1_security_information_smart security_passenger,"[security, information, smart security, passen...",[Smart Security Smart Security is joint initia...
3,2,170,2_settlement_billion_processed_bsp,"[settlement, billion, processed, bsp, cass, is...",[IATA Cargo Account Settlement System (CASS) s...
4,3,146,3_charges_airport_airports_infrastructure,"[charges, airport, airports, infrastructure, c...",[Airports Savings in airport charges of $1.5 b...
5,4,143,4_global_governments_important_great,"[global, governments, important, great, future...",[Annual Review 2021\n\nWillie Walsh Director G...
6,5,92,5_ndc_order_travel_retailing,"[ndc, order, travel, retailing, distribution, ...",[With passenger volume expected to double by 2...
7,6,90,6_baggage_fast travel_fast_boarding,"[baggage, fast travel, fast, boarding, passeng...",[The program provides self-service options at ...
8,7,89,7_cargo_e-freight_freight_e-awb,"[cargo, e-freight, freight, e-awb, chain, supp...","[In partnership with shippers, freight forward..."
9,8,84,8_saf_corsia_emissions_production,"[saf, corsia, emissions, production, carbon, i...",[Sustainable aviation fuels One of the most pr...


In [4]:
# ------------------------------------------------------------
# Visualize intertopic distances
# (move this to a function)
# ------------------------------------------------------------

# N.B. UMAP (Uniform Manifold Approximation and Projection)
# used for dimensionality reduction. 
fig_2d = topic_model.visualize_topics(custom_labels=True)

# Update background and grid
fig_2d.update_layout(
    plot_bgcolor="white",
    paper_bgcolor="white",
    font_family="Arial",
    title={
        "text": f"Intertopic Distance Map<br><sup>{model_label}</sup>",
        "x": 0.15,
        "font": {"size": 20}
    },
    width=1000,
    height=800,
    xaxis=dict(
        showgrid=False,
        zeroline=True,
        showline=True,
        ticks="outside",
        title=None
    ),
    yaxis=dict(
        showgrid=False,
        zeroline=True,
        showline=True,
        ticks="outside",
        title=None
    ),
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="center",
        x=0.5
    )
)
fig_2d.write_html("../outputs/intertopic_distance_map_2D.html")   

fig_2d.show()

In [5]:
# ------------------------------------------------------
# Clean up labels and remove junk topics
# ------------------------------------------------------

# Retrieve the current model labels before overwriting
old_labels = topic_model.topic_labels_

# Generate new labels
clean_labels, junk_topics = generate_clean_labels(topic_model, top_n_words=1)

# Print label changes for review
print("\nGenerated topic labels:")
for topic_id, label in sorted(clean_labels.items()):
    old_label = old_labels.get(topic_id)
    print(f"Topic {topic_id}: {label} (OLD: {old_label})")

# Apply to the model
topic_model.set_topic_labels(clean_labels)

Topic 23 identified as junk:
[('', 1e-05), ('', 1e-05), ('', 1e-05), ('', 1e-05), ('', 1e-05), ('', 1e-05), ('', 1e-05), ('', 1e-05), ('', 1e-05), ('', 1e-05)]

Generated topic labels:
Topic -1: NULL (OLD: -1_fuel_emissions_passenger_global)
Topic 0: Safety (OLD: 0_safety_iosa_accidents_accident)
Topic 1: Security (OLD: 1_security_information_smart security_passenger)
Topic 2: Settlement/BSP (OLD: 2_settlement_billion_processed_bsp)
Topic 3: Airport Charges/Infrastructure (OLD: 3_charges_airport_airports_infrastructure)
Topic 4: Global Governments (OLD: 4_global_governments_important_great)
Topic 5: NDC (OLD: 5_ndc_order_travel_retailing)
Topic 6: Baggage (OLD: 6_baggage_fast travel_fast_boarding)
Topic 7: Cargo & E-Freight (OLD: 7_cargo_e-freight_freight_e-awb)
Topic 8: SAF (OLD: 8_saf_corsia_emissions_production)
Topic 9: Regulation (OLD: 9_regulation_rights_passenger rights_regulations)
Topic 10: Markets Growth (OLD: 10_markets_growth_load_premium)
Topic 11: COVID‑19 (OLD: 11_covid-

In [6]:
# ------------------------------------------------------------
# Generate topic time series and visualize trends
# ------------------------------------------------------------

timestamps = pd.to_datetime(timestamps, format="%Y")
topics_over_time = topic_model.topics_over_time(texts, timestamps, topic_model.topics_)

# OPTIONAL: Set filter criteria
# Set to None to ignore
min_year = None
max_year = None
exclude_years = None #[2020]  

# Apply year filtering
filtered_topics, filter_label = filter_dataframe_by_year(
    topics_over_time,
    min_year=min_year,
    max_year=max_year,
    exclude_years=exclude_years
)

# Apply topic filtering (remove junk and noise topics)
filtered_topics = filtered_topics[
    (~filtered_topics["Topic"].isin(junk_topics)) &
    (filtered_topics["Topic"] != -1)
]

# Select top k most frequent clean topics
top_k = 10
top_topics = get_top_topics(filtered_topics, top_k)


# Make plot of topic frequency by year
# y-axis = % of all text chunks assigned to this topic in a given year
fig = topic_model.visualize_topics_over_time(filtered_topics, topics=top_topics)

# Apply custom colors and labels
palette = px.colors.qualitative.T10

for i, trace in enumerate(fig.data):
    topic_id = top_topics[i]
    trace.line.color = palette[i % len(palette)]
    trace.name = clean_labels[topic_id]
    hover_text = trace.hovertemplate or ""
    trace.hovertemplate = f"{clean_labels[topic_id]}<extra></extra>"

# Clean up the figure layout
fig.update_layout(
    title=f"Topic Trends Over Time<br><sup>{filter_label}</sup>",
    width=1200,
    height=600,
    xaxis_title="Year",
    yaxis_title="Topic Frequency (% of Annual Report)",
    legend_title="Topics",
    font=dict(family="Arial", size=12),
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=-0.4,
        xanchor="center",
        x=0.5
    )
)

output_filename = f"../outputs/topic_trends"
fig.write_html(f"{output_filename}.html")
fig.write_image(f"{output_filename}.png")

fig.show()

19it [00:05,  3.45it/s]


In [7]:
# ===============================================
# Generate a table matching trends to business
# opportunities, and assign a priority ranking
# based on trend size and growth
# ===============================================

#matched_opportunities_df, unmatched_topics = generate_opportunity_table(topic_model, topics_over_time)
#matched_opportunities_df, unmatched_topics = generate_opportunity_table(topic_model, filtered_topics)
matched_opportunities_df, unmatched_topics = generate_opportunity_table(topic_model, filtered_topics, clean_labels=clean_labels)

print(f"Filtering (if any): {model_label}")
print(f"Model Name: {model_name}\n")
print("=== Matched Business Opportunities ===\n")
display(matched_opportunities_df)

# Save table as CSV
output_filename = f"../outputs/business_opportunities.csv"
matched_opportunities_df.to_csv(output_filename, index=False)

print(f"\nSaved opportunity table to: {output_filename}")

Filtering (if any): 
Model Name: bertopic_cluster_model

=== Matched Business Opportunities ===



,Trend,Detected Label,Implication,Ongoing Activity,Priority (Recommended),Keyword Fraction (%),Topic Growth
0,Safety,0_safety_iosa_accidents_accident,Ongoing emphasis on flight and operational saf...,"Invest in IOSA certification, risk management,...",High,9.62,7
1,Airport Charges/Infrastructure,3_charges_airport_airports_infrastructure,Rising airport and regulatory charges impactin...,Advocate for transparency and collaborate on c...,High,6.03,6
2,SAF,8_saf_corsia_emissions_production,Industry push for sustainable fuels to meet ne...,Invest in SAF partnerships and explore supply ...,High,3.47,15
3,Security,1_security_information_smart security_passenger,Need for secure and seamless passenger identit...,Adopt biometric identity systems and enhance c...,Medium,8.01,4
4,Cargo & E-Freight,7_cargo_e-freight_freight_e-awb,Cargo remains a strong revenue pillar post-pan...,Expand dedicated cargo capacity and invest in ...,Medium,3.68,2
5,Markets Growth,10_markets_growth_load_premium,Shifting demand patterns and regional recovery...,Refocus network planning on high-growth and un...,Medium,2.93,1
6,Tax,13_tax_taxes_taxation_government,Growing tax pressures from governments on avia...,Strengthen policy advocacy and evaluate operat...,Medium,2.73,2
7,Cargo,15_cargo_trade_growth_global,Macro-level trends influencing aviation growth...,"Adapt strategy to respond to geopolitical, eco...",Medium,2.48,5
8,Settlement/BSP,2_settlement_billion_processed_bsp,Evolution of industry financial frameworks (e....,Upgrade payment systems and optimize cash flow...,Low,7.02,-1
9,Global Governments,4_global_governments_important_great,Macro-level trends influencing aviation growth...,"Adapt strategy to respond to geopolitical, eco...",Low,5.91,-7



Saved opportunity table to: ../outputs/business_opportunities.csv
